In [58]:
include("src/inversionModel.jl")
include("src/rejectionModel.jl")
include("src/rangeModel.jl")
include("src/linearModel.jl")
include("src/config.jl")
include("src/crustDistribution.jl")

Loading crust information from file resources/crustDistribution.jld


Loading crust information from file resources/crustDistribution.jld


Loading crust information from file resources/crustDistribution.jld


Main.crustDistribution

In [59]:
models = makeModels("huang", modelType=LinearModel);

Building models100%|████████████████████████████████████| Time: 0:00:12


In [60]:
"""
  getTestSeismic()
Build a test earth directly from model seismic properties: so model contains superset
of rocks in the test earth
"""
function getTestSeismic(n, models::ModelCollection, target::Float64, layer::String; nsamples=100)
    # compositions to choose from 
    comps = models.models[layer][1].comp
    indices = Array(1:size(comps,1))
    sio2 = comps[:,2]
    
    # choose comp samples from ign of model 1 that average to target
    s = sample(indices,nsamples, replace=false)
    snew = zeros(Int, length(s))
    snew .= s
    best = abs(mean(sio2[s]) - target)
    while best > 1/100
      to_replace = sample(1:nsamples)
      new = sample(indices)
      snew[to_replace] = new
      if abs(mean(sio2[snew]) - target) < best
          s .= snew
          best = abs(mean(sio2[snew]) - target)
          #println("new best $(mean(sio2[snew]))")
      end
      snew .= s # reset
    end
    s = Int.(s)
    # sio2 comp
    println(mean(comps[s,2], dims=1))
    
    # n geotherm samples
    crust = crustDistribution.getCrustParams(n, uncertain=true) # depth to 550, base of upper, middle, lower
    geotherm = crust[:,1]
    bins = [searchsortedlast(Array(models.bins)[1:end-1], g) for g in geotherm]
    println(bins)
    
    # assign comp to each of n
    assignment = sample(s, n)
    println(assignment)
    
    # for each of n, choose the seismic props from the model with the correct geotherm bin
    seismic = zeros(n,3)
    for i in 1:n
        seismic[i,:] .= models.models[layer][bins[i]].seismic[assignment[i],2:end]
    end
    return (seismic[:,1], seismic[:,2], seismic[:,3], geotherm)
end

getTestSeismic

In [61]:
getTestSeismic(1000, models, 60.0, "upper")

LoadError: BoundsError: attempt to access 19739-element Vector{Float64} at index [[13307, 1681, 314, 17457, 4425, 19097, 7743, 3633, 2068, 19666  …  16447, 8946, 3190, 6878, 10570, 793, 15662, 11030, 3823, 6445]]

In [44]:
searchsortedlast(models.bins, 23)

1

In [77]:
comps = models.models["upper"][1].comp
indices = Array(1:size(comps,1))
sio2 = comps[:,2]

s = sample(indices,100, replace=false)
println(mean(sio2[s]))

51.936312099999995


In [73]:
s[s .> 19739]

Int64[]